In [ ]:
import sys
from io import StringIO
from IPython import get_ipython

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        print("exiting")
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup

def ipy_exit():
    raise IpyExit

-------------------------------------
## Prepare Source, Data and Directories
- set input/output directories and user settings
- download source from github
- replace data directory with precomputed one

### **REQUIREMENTS:**
   - you have joined the project `covid19dynstat`
   - you have created the directory `$PROJECT_covid19dynstat/$USER`
   - you have copied/linked a `data` directory to `$PROJECT_covid19dynstat/$USER/data`
       - will replace `data` from git repository
       - MUST include `counties/counties.pkl` (explicitly for JURECA, not from git repo!)

In [ ]:
# fixed parameters
user_email = "user@email.de"
cmpprj_name = "covid19dynstat" # do not change (used in slurm-file)

# run:
run_name = "BSTIM-Covid19_001"

# source: clone only first time with 'run_name'
git_repo = "https://github.com/neuroinfo-os/BSTIM-Covid19.git"
git_branch = "master"

# csv: update if missing or requested
csv_rki = True # get csv from RKI
csv_update = False # update csv even if already existing

# run slurm dependency chain
slurm_chain = False

# simulation: sample_ia
# nodes * taskpernode = num_sample defined in src/sample_ia_effects.py
submit_job_ia   = True
job_ia_nodes        = 5 # 5
job_ia_taskspernode = 20 # 20
job_ia_runtime      = '5:00:00' # '5:00:00' (JURECA + 1h)

# simulation: sample_posterior
# nodes * taskpernode = len(combinations) defined in src/config.py
submit_job_post   = False
job_post_nodes        = 1 # 1
job_post_taskspernode = 4 # 4
job_post_runtime      = '18:00:00' # '18:00:00' (JURECA + 1h)

### Set input/output directories and user settings

In [ ]:
import os

# check users project directory
# (do NOT change as it is used in SLURM-jobscript, too)
user_proj_dir = os.path.join(os.getenv('PROJECT_'+cmpprj_name), os.getenv('USER'))
if not os.path.isdir(user_proj_dir):
    print("ERROR: project directory has no directory of the current user.")
    print("       please create ", user_proj_dir)
    raise IpyExit
else:
    print("user_proj_dir = ", user_proj_dir)
    
# check for SLURM output directory
# (do not change as it is set in SLURM-jobscript, too)
slurm_out_dir = os.path.join(user_proj_dir, 'run')
if not os.path.isdir(user_proj_dir):
    print("ERROR: SLURM output directory does not exist.")
    print("       please create ", slurm_out_dir)
    raise IpyExit
else:
    print("slurm_out_dir = ", slurm_out_dir)

### Download source 

In [ ]:
import os
from datetime import datetime
from git import Repo

# auto-generate run_name if empty
if not run_name:
    run_name = datetime.now().strftime("BSTIM-Covid19_%Y-%m-%d_%H-%M-%S_" + git_branch)

# set source_dir
source_dir = os.path.join(user_proj_dir, run_name)
print("source_dir = ", source_dir)

# set source_data_dir
source_data_dir = os.path.join(source_dir, 'data')
print("source_data_dir = ", source_data_dir)

# clone source if required
if not os.path.exists(source_dir):
    print("cloning git repository")

    if not git_branch:
        git_branch = 'master'
    print("git_branch = ", git_branch)

    if not git_repo:
        git_repo = 'https://github.com/neuroinfo-os/BSTIM-Covid19.git'
    print("git_repo = ", git_repo)

    repo = Repo.clone_from(git_repo, source_dir, branch=git_branch)
    #repo.heads['tag-name'].checkout() # checkout a certain tag

    # rename data dir if from clean checkout
    if os.path.exists(source_data_dir):
        os.rename(source_data_dir,  os.path.join(source_dir,'data_git'))
else:
    print("NO cloning of git repository")

### Check requirements

In [ ]:
import sys
import pkg_resources
from pkg_resources import DistributionNotFound, VersionConflict

source_reqtxt_file = os.path.join(source_dir, 'requirements.txt')
if os.path.exists(source_reqtxt_file):
    with open(source_reqtxt_file) as f:
        dependencies = f.readlines()
    dependencies = [x.strip() for x in dependencies] 

    for d in dependencies:
        if d == "pkg-resources==0.0.0":
            continue
        try:
            pkg_resources.require(d)
        except pkg_resources.DistributionNotFound as e:
            print("MISSING package: ", e)
        except pkg_resources.VersionConflict as e:
            print("WRONG version: ", e)
else:
    print("requirements.txt not found at ", source_reqtxt_file)

### Replace data directory with precomputed one

In [ ]:
import os
import shutil

# copy new data dir with precomputed pickel file
# if data directory does not exist
if not os.path.exists(source_data_dir):
    print("replacing data directory")
    user_data_dir = os.path.join(user_proj_dir, 'data')
    if os.path.exists(user_data_dir) and os.path.isdir(user_data_dir):
        shutil.copytree(user_data_dir, source_data_dir)
    else:
        print("ERROR: user data directory does not exists or is no directory.")
        raise IpyExit
else:
    print("NO replacing of data directory")

-------------------------------------------------------
## Get COVID-19 data from RKI
Covid-19 data is provided by the [Robert Koch Institute][4] via the publically accessiable [this](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0/data?orderBy=Meldedatum) link.  
We download the CSV table and store it in `./data/raw/`

### Copy COVID-19 data from data-git (if csv_rki == False)

In [ ]:
import os
import shutil

raw_csv_fpath = os.path.join(source_data_dir, 'raw', "covid19.csv")
print("raw_csv_fpath = ", raw_csv_fpath)

data_basename = os.path.basename(raw_csv_fpath)
datagit_csv_fpath = os.path.join(source_dir, "data_git", "diseases", data_basename)
if os.path.exists(datagit_csv_fpath):
    print("datagit_csv_fpath = ", datagit_csv_fpath)
    
    data_csv_fpath = os.path.join(source_dir, "data", "diseases", data_basename)
    print("data_csv_fpath = ", data_csv_fpath)

    # only if update is requested
    if os.path.exists(data_csv_fpath) and csv_update:
        
        # copy csv from data_git instead of downloading
        if not csv_rki:
            shutil.copyfile(datagit_csv_fpath, data_csv_fpath)
            print("copy", data_basename, "from git ... SUCCESS")
        else:
            print("NO copying - download from RKI requested")
    else:
        print("NO csv update")
else:
    print("data_csv_fpath = MISSING")

### Download COVID-19 CSV (if csv_rki == True)

In [ ]:
import os
import requests
from datetime import datetime

# download csv data if asked for
if csv_rki and not os.path.exists(raw_csv_fpath) or csv_update:
    print('download COVID-19 data')

    raw_csv_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv'
    raw_csv_red = requests.get(raw_csv_url, allow_redirects=True)
    print("raw_csv_url = ", raw_csv_url)

    open(raw_csv_fpath, 'wb').write(raw_csv_red.content)
else:
    print("NO download of COVID-19 data")

### Preprocess COVID-19 CSV (if csv_rki == True)
The downloadable CSV table can be found in `./data/raw/` and is preprocessed to fit the BSTI Model implementation.

In [ ]:
data_csv_fpath = os.path.join(os.path.dirname(raw_csv_fpath), "..", "diseases", os.path.basename(raw_csv_fpath))
print("data_csv_fpath = ", data_csv_fpath)

# skip preprocessiong
#if not csv_rki or os.path.exists(data_csv_fpath) and not csv_update:
#    if os.path.exists(data_csv_fpath) or not csv_update:
#        print("NO preprocessing csv")
#        raise StopExecution

# do preprocessing
preprocess_python = os.path.join(source_dir, 'src', "preprocess_covid19_table.py")
if not os.path.exists(preprocess_python):
    print("ERROR: cannot find ", preprocess_python)
    raise IpyExit
print("preprocess_python = ", preprocess_python)

data_shapes_fpath = os.path.join(os.path.dirname(raw_csv_fpath), 'germany_county_shapes.json')
if not os.path.exists(data_shapes_fpath):
    print("ERROR: cannot find ", data_shapes_fpath)
    raise IpyExit
print("data_shapes_fpath = ", data_shapes_fpath)    

regex_Meldedatum = r'([0-9]{4})\/([0-9]{2})\/([0-9]{2}).*'
dayformat_Meldedatum = r'{:04d}/{:02d}/{:02d} 00:00:00'
%run $preprocess_python --source "$raw_csv_fpath" --destination "$data_csv_fpath" --shapes "$data_shapes_fpath" --regex "$regex_Meldedatum" --dayformat "$dayformat_Meldedatum"
print(" finished")

total = 0        
with open(data_csv_fpath) as fin:
    reader = csv.reader(fin)
    next(reader, None)  # skip the headers
    for row in reader:
        total += sum(map(int, row[1:]))
    print("infections = ", total)
    if total <= 0:
        print("ERROR: csv data probably wrong")
        raise IpyExit

### Count infections

In [ ]:
import os
import csv

if not os.path.isfile(data_csv_fpath):
    print("ERROR: file data/covid19.csv is missing")
    raise IpyExit
        
total = 0
with open(data_csv_fpath) as fin:
    reader = csv.reader(fin)
    next(reader, None)  # skip the headers
    for row in reader:
        total += sum(map(int, row[1:]))
    print("infections = ", total)
    if total <= 0:
        print("ERROR: csv data probably wrong")
        raise IpyExit

----------------------------------------
## Simulation 1: Sample Interaction Effects (100x)
`sample_ia_effects.py` reads  
- ../data/counties/counties.pkl
- ../data/diseases/{covid19}.csv

and outputs:
- ../data/ia_effect_samples/{}_{}.pkl

sample_ia_effects.py is called 100x by SLURM (=farming) and calculates the same thing with different random numbers:  
- gridjob_sample_ia.slurm 
  - `#SBATCH --array=1-100:4`
- gridjob_sample_ia.slurm.sh
  - `THEANO_FLAGS="base_compiledir=${TASK_DIR}/,floatX=float32,device=cpu,openmp=True,mode=FAST_RUN,warn_float64=warn" OMP_NUM_THREADS=8  python3 sample_ia_effects.py > ${TASK_DIR}/log.txt`
- sample_ia_effects.py
  - ```nums_sample = range(100)
GID = int(os.environ["SGE_TASK_ID"])
num_sample = nums_sample[GID - 1]
filename = "../data/ia_effect_samples/{}_{}.pkl".format(disease, num_sample)```

### Check requirements

In [ ]:
import os
import csv

# slurm jobfile
slurm_jobfile_ia = os.path.join(source_dir, 'src', "gridjob_sample_ia.slurm")
if not os.path.exists(slurm_jobfile_ia):
    print("ERROR: original SLURM job file does not exist")
    print("       please check for ", slurm_jobfile_ia)
    raise IpyExit
print("slurm_jobfile_ia = ", slurm_jobfile_ia)
    
slurm_shfile_ia = os.path.join(source_dir, 'src', "gridjob_sample_ia.slurm.sh")
if not os.path.exists(slurm_shfile_ia):
    print("ERROR: original SLURM sh file does not exist")
    print("       please check for ", slurm_shfile_ia)
    raise IpyExit
print("slurm_shfile_ia = ", slurm_shfile_ia)
    
# check is shell-script has executable bit
if not os.access(slurm_shfile_ia, os.X_OK):
    with open(slurm_shfile_ia, "r") as fd:
        os.chmod(fd.fileno(), 0o755)

if submit_job_ia:
    
    # check required input
    file_counties = os.path.join(source_data_dir, 'counties', 'counties.pkl')
    if not os.path.isfile(file_counties):
        print("ERROR: file data/counties.pkl is missing")
        raise IpyExit
    else:
        print("file_counties = ", file_counties)
        
    file_covid19 = os.path.join(source_data_dir, 'diseases', 'covid19.csv')
    if not os.path.isfile(file_covid19):
        print("ERROR: file data/covid19.csv is missing")
        raise IpyExit
    else:
        total = 0        
        with open(file_covid19) as fin:
            reader = csv.reader(fin)
            next(reader, None)  # skip the headers
            for row in reader:
                total += sum(map(int, row[1:]))
        print("infections = ", total)
        if total <= 0:
            print("ERROR: csv data probably wrong")
            raise IpyExit
    print("file_covid19 = ", file_covid19)
    
    # check output directory
    dir_iaeffect = os.path.join(source_data_dir, 'ia_effect_samples')
    if not os.path.exists(dir_iaeffect):
        os.mkdir(dir_iaeffect)    
    if not os.path.isdir(dir_iaeffect):
        print("ERROR: " + dir_iaeffect + " not a directroy")
        raise IpyExit
    if os.listdir(dir_iaeffect):
        print("ERROR: directory " + dir_iaeffect + " not empty")
        raise IpyExit
    print("dir_iaeffect = ", dir_iaeffect)
    
else:
    print("NO checking")

### Prepare SLURM job file (sample_ia)

In [ ]:
import fileinput

def prepare_jobfile_ia(slurm_jobfile_ia):
    
    # replace lines with user-info
    finput = fileinput.input(slurm_jobfile_ia, inplace=1)
    try:
        for line in finput:
            line = "#SBATCH --array=1-{}:{}\n".format(job_ia_nodes * job_ia_taskspernode, job_ia_taskspernode) if "#SBATCH --array=" in line else line
            line = "#SBATCH --ntasks-per-node={}\n".format(job_ia_taskspernode) if "#SBATCH --ntasks-per-node=" in line else line
            line = "#SBATCH --nodes=1\n"                        if "#SBATCH --nodes="       in line else line
            line = "#SBATCH --time={}\n".format(job_ia_runtime) if "#SBATCH --time="        in line else line
            line = "#SBATCH --mail-type=ALL\n"                  if "# #SBATCH --mail-type=" in line else line
            line = "#SBATCH --mail-user=" + user_email + '\n'   if "# #SBATCH --mail-user=" in line else line
            line = "# mkdir -p ${PROJECT}/${USER}/runs/\n" if "mkdir -p ${PROJECT}/${USER}/runs/" in line else line
            line = "srun -n ${SLURM_NTASKS} gridjob_sample_ia.slurm.sh" if "srun --exclusive -n ${SLURM_NTASKS} gridjob_sample_ia.slurm.sh\n" in line else line
            print(line, end='')
    except:
        print("ERROR: could not prepare slurm job file")
        raise IpyExit
    finput.close()

if submit_job_ia:
    prepare_jobfile_ia(slurm_jobfile_ia)
    with open(slurm_jobfile_ia, 'r') as sfile:
        print(sfile.read())
else:
    print("NO submit of slurm ia job")

### Submit SLURM job (sample_ia)

In [ ]:
import os
import time
from subprocess import (Popen, PIPE)

def submit_job(slurm_jobfile, submit_dir, sbatch_addargs=''):
    print("submitting slurm job")
    
    # build sbatch command
    sbatch_args = sbatch_addargs + " " + slurm_jobfile
    sbatch_cmd = ['sbatch'] + sbatch_args.split()
    print("sbatch_cmd = ", sbatch_cmd)

    # submit SLURM job
    process = Popen(sbatch_cmd, stdin=PIPE, stdout=PIPE, stderr=PIPE, cwd=submit_dir)
    
    # block until finished and output stdout, stderr
    stdout, stderr = process.communicate() 
    sbatch_out = stdout.decode("utf-8")
    sbatch_err = stderr.decode("utf-8")
    
    print("---- stdout ----")        
    print(sbatch_out)
    print("---- stderr ----")
    print(sbatch_err)
        
    if process.returncode != 0:
        raise IpyExit
    
    # get SLURM job id
    slurm_jobid = ''
    if sbatch_out:
        slurm_jobid = sbatch_out.split()[-1]
    print("slurm_jobid = ", slurm_jobid)

    # save SLURM job id to file
    if slurm_jobid:
        with open(os.path.join(submit_dir, slurm_jobfile + ".sbatchout"), "w") as ofile:
            print("jobid: {}".format(slurm_jobid), file=ofile)

submit_dir_ia = os.path.join(source_dir, 'src')
print("submit_dir_ia = ", submit_dir_ia)

if submit_job_ia:
    submit_job(slurm_jobfile_ia, submit_dir_ia, '-vv')
    
    # better wait for a few seconds to ensure slurm has processed the new job internally
    # this ensures, that squeue will show at least some information
    time.sleep(5)
else:
    print("NO submit of slurm ia job")

### Show status of submitted job (sample_ia)

In [ ]:
from subprocess import (Popen, PIPE)

def status_job(slurm_jobid):
                  
    # build squeue command
    squeue_args = ' -l -u ' + os.getenv('USER') + ' -j ' + slurm_jobid
    squeue_cmd = ['squeue'] + squeue_args.split()
    print("squeue_cmd = ", squeue_cmd)

    # show status
    squeue_out = ''
    process = Popen(squeue_cmd, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    if stderr:
        print(stdout.decode("utf-8"))
        print(stderr.decode("utf-8"))
        #raise IpyExit
    return stdout.decode("utf-8")

def get_slurm_jobid(sbatchout_file):

    # get slurm job id from file
    if os.path.exists(sbatchout_file):
        with open(sbatchout_file) as ifile:
            for line in ifile:
                if 'jobid:' in line:
                    slurm_jobid = line.split()[-1]
                    print("slurm_jobid = ", slurm_jobid)
                    return slurm_jobid
    else:
        print("ERROR: not found " + sbatchout_file)

# get job id
sbatchout_file_ia = os.path.join(submit_dir_ia, slurm_jobfile_ia + ".sbatchout")
slurm_jobid_ia = get_slurm_jobid(sbatchout_file_ia)

# call 'squeue'
if slurm_jobid_ia:            
    squeue_out_ia = status_job(slurm_jobid_ia)
    print(squeue_out_ia)
else:
    print("NO submit of slurm ia job")

### Check output directory (sample_ia)

In [ ]:
import os

# get job id
sbatchout_file_ia = os.path.join(submit_dir_ia, slurm_jobfile_ia + ".sbatchout")
slurm_jobid_ia = get_slurm_jobid(sbatchout_file_ia)

if not slurm_chain:

    # SCRATCH
    scratch_dir = os.getenv("SCRATCH_" + cmpprj_name)
    if slurm_jobid_ia and scratch_dir:
        job_scratchdir_ia = os.path.join(scratch_dir, "run_" + slurm_jobid_ia)
        print("job_scratchdir_ia = ", job_scratchdir_ia)
        if not os.path.exists(job_scratchdir_ia):
            print("scratch directory missing - perhaps the job has not started yet")
        #%ls -la $job_scratchdir_ia

    # data output
    job_outdir_ia = os.path.join(source_data_dir, "ia_effect_samples")
    print("job_outdir_ia = ", job_outdir_ia)
    if not os.path.exists(job_outdir_ia):
        print("output directory mission - perhaps the job has not started yet")
        raise IpyExit
    #%ls -la $job_outdir_ia

----------------------------------------
## Simulation 2: Sample Posterior (4x)
`sample_posterior.py` reads  
- ./data/counties/counties.pkl
- ./data/ia_effect_samples/covid19_{}.pkl

and outputs:
- ../data/mcmc_samples_backup/

sample_posterior.py is called 4x by SLURM for add combinations defined in src/config.py:
- gridjob_sample_posterior.slurm 
  - `#SBATCH --array=1-4:8`
- gridjob_sample_posterior.slurm.sh
  - `THEANO_FLAGS="base_compiledir=${TASK_DIR}/,floatX=float32,device=cpu,openmp=True,mode=FAST_RUN,warn_float64=warn" python3 sample_posterior.py > ${TASK_DIR}/log.txt`
- sample_posterior.py
  - ```i = int(os.environ["SGE_TASK_ID"])-1
model_complexity, disease = combinations[i]
filename_params = "../data/mcmc_samples_backup/parameters_{}_{}_{}".format(disease, use_interactions, use_report_delay)
filename_pred = "../data/mcmc_samples_backup/predictions_{}_{}_{}.pkl".format(disease, use_interactions, use_report_delay)
filename_model = "../data/mcmc_samples_backup/model_{}_{}_{}.pkl".format(disease, use_interactions, use_report_delay)```

### Check requirements

In [ ]:
import os

# check for SLURM job file
slurm_jobfile_post = os.path.join(source_dir, 'src', "gridjob_sample_posterior.slurm")
if not os.path.exists(slurm_jobfile_post):
    print("ERROR: original SLURM job file does not exist")
    print("       please check for ", slurm_jobfile_post)
    raise IpyExit
else:
    print("slurm_jobfile_post = ", slurm_jobfile_post)

slurm_shfile_post = os.path.join(source_dir, 'src', "gridjob_sample_posterior.slurm.sh")
if not os.path.exists(slurm_shfile_post):
    print("ERROR: original SLURM sh file does not exist")
    print("       please check for ", slurm_shfile_post)
    raise IpyExit
print("slurm_shfile_ia = ", slurm_shfile_post)
    
# check is shell-script has executable bit
if not os.access(slurm_shfile_post, os.X_OK):
    with open(slurm_shfile_post, "r") as fd:
        os.fchmod(fd.fileno(), 0o755)

if submit_job_post:
    
    # check required input
    file_counties = os.path.join(source_data_dir, 'counties', 'counties.pkl')
    if not os.path.isfile(file_counties):
        print("ERROR: file data/counties.pkl is missing")
        raise IpyExit
    else:
        print("file_counties = ", file_counties)

    if not slurm_chain:
        # check output of sample_ia (only if we slurm dependencies)
        dir_iaeffect = os.path.join(source_data_dir, 'ia_effect_samples')
        for i in range(100):
            file_iaeffect = os.path.join(dir_iaeffect, "covid19_{}.pkl".format(i))
            if not os.path.isfile(file_iaeffect):
                print("ERROR: file " + file_iaeffect + "  is missing")
                raise IpyExit
        print("file_iaeffect = " + dir_iaeffect + "/covid19_{}.pkl")
    
    # check output directory
    dir_mcmcsamples = os.path.join(source_data_dir, 'mcmc_samples_backup')
    if not os.path.exists(dir_mcmcsamples):
        os.mkdir(dir_mcmcsamples)
    if not os.path.isdir(dir_mcmcsamples):
        print("ERROR: " + dir_mcmcsamples + " not a directory")
        raise IpyExit
    if os.listdir(dir_mcmcsamples):
        print("ERROR: directory " + dir_mcmcsamples + " not empty")
        raise IpyExit
    print("dir_mcmcsamples = ", dir_mcmcsamples)

### Prepare SLURM job file (sample_posterior)

In [ ]:
import fileinput

def prepare_jobfile_post(slurm_jobfile_post):
    
    # replace lines with user-info
    finput = fileinput.input(slurm_jobfile_post, inplace=1)
    try:
        for line in finput:
            line = "#SBATCH --array=1-{}:{}\n".format(job_post_nodes * job_post_taskspernode, job_post_taskspernode) if "#SBATCH --array=" in line else line
            line = "#SBATCH --ntasks-per-node={}\n".format(job_post_taskspernode) if "#SBATCH --ntasks-per-node=" in line else line
            line = "#SBATCH --nodes=1\n"                          if "#SBATCH --nodes="       in line else line
            line = "#SBATCH --time={}\n".format(job_post_runtime) if "#SBATCH --time="        in line else line
            line = "#SBATCH --mail-type=ALL\n"                    if "# #SBATCH --mail-type=" in line else line
            line = "#SBATCH --mail-user=" + user_email + '\n'     if "# #SBATCH --mail-user=" in line else line
            line = "# mkdir -p ${PROJECT}/${USER}/runs/\n" if "mkdir -p ${PROJECT}/${USER}/runs/" in line else line
            line = "srun -n ${SLURM_NTASKS} gridjob_sample_posterior.slurm.sh" if "srun --exclusive -n ${SLURM_NTASKS} gridjob_sample_posterior.slurm.sh\n" in line else line
            print(line, end='')
    except:
        print("ERROR: could not prepare slurm job file")
        raise IpyExit
    finput.close()

if submit_job_post:
    prepare_jobfile_post(slurm_jobfile_post)
    with open(slurm_jobfile_post, 'r') as sfile:
        print(sfile.read())
else:
    print("NO submit of slurm ia job")

### Submit SLURM job (sample_posterior)

In [ ]:
submit_dir_post = os.path.join(source_dir, 'src')
print("submit_dir_post = ", submit_dir_post)

if submit_job_post:
    
    sbatch_addargs = '-vv'

    if slurm_chain:
        sbatchout_file_ia = os.path.join(submit_dir_ia, slurm_jobfile_ia + ".sbatchout")
        slurm_jobid_ia = get_slurm_jobid(sbatchout_file_ia)
        if slurm_jobid_ia:
            sbatch_addargs += " --dependency=aftercorr:{}".format(slurm_jobid_ia)
    
    submit_job(slurm_jobfile_post, submit_dir_post, sbatch_addargs)
    
    # better wait for a few seconds to ensure slurm has processed the new job internally
    # this ensures, that squeue will show at least some information
    time.sleep(5)
else:
    print("NO submit of slurm job")

### Show status of submitted job (sample_posterior)

In [ ]:
# get job id
sbatchout_file_post = os.path.join(submit_dir_post, slurm_jobfile_post + ".sbatchout")
slurm_jobid_post = get_slurm_jobid(sbatchout_file_post)

# call 'squeue'
if slurm_jobid_post:
    squeue_out_post = status_job(slurm_jobid_post)
    print(squeue_out_post)
else:
    print("ERROR: no slurm job id found")